In [ ]:
image, angle = LoadImg('')



device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
Net = torchvision.models.resnet18(pretrained=True) # Load net
Net.fc = torch.nn.Linear(in_features=512, out_features=1, bias=True) # Set final layer to predict one value
Net = Net.to(device) # Assign net to gpu or cpu

modelPath = '1000.torch'
Net.load_state_dict(torch.load(modelPath))

#Net.eval() # Set net to evaluation mode, usually usefull in this case its fail


Img = torch.autograd.Variable(Img, requires_grad=False).to(device).unsqueeze(0) # Convert to pytorch

with torch.no_grad():
    Prd = Net(Img)  # Run net

print("Predicted Steering Angle: ", Prd.data.cpu().numpy())
print("Real Steering Angle: ", Angle)